# Tutorial de Autenticação com Flask-Login

Este guia detalhado vai te ensinar a proteger rotas e gerenciar o acesso de usuários em sua aplicação Flask, usando a extensão Flask-Login.

### 1\. Instalação e Configuração

Para começar, você precisa instalar a extensão `Flask-Login` e o `Werkzeug`, que é uma biblioteca para criptografia de senhas.

1.  **Instale as Extensões:**

      * Abra o terminal do PyCharm.
      * Execute os seguintes comandos para instalar as bibliotecas necessárias:
        ```bash
        pip install Flask-Login Werkzeug
        ```

2.  **Salve as Dependências:**

      * Salve as novas dependências no seu arquivo `requirements.txt`:
        ```bash
        pip freeze > requirements.txt
        ```

3.  **Configure o `Flask-Login`:**

      * Abra seu arquivo `app.py` e adicione o código de configuração.
      * Você precisará de uma chave secreta para a sua aplicação e para o gerenciador de login. Use sempre o `app.config` para definir as configurações.

    **`app.py`**

In [ ]:
from flask import Flask, render_template, redirect, url_for, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.config['SECRET_KEY'] = 'uma-chave-secreta-forte'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login' # Define a rota para o login
login_manager.login_message = 'Por favor, faça login para acessar esta página'
login_manager.login_message_category = 'warning'

# Funcao de carregamento do usuario para o Flask-Login
@login_manager.user_loader
def load_user(user_id):
    return Usuario.query.get(int(user_id))

# Classe de modelo do Usuario
class Usuario(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    nome = db.Column(db.String(50), nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)
    senha = db.Column(db.String(60), nullable=False)

    def __repr__(self):
        return f"Usuario('{self.nome}', '{self.email}')"

with app.app_context():
    db.create_all()

A classe `Usuario` herda de `UserMixin`, que fornece implementações padrão para propriedades e métodos que o Flask-Login espera, como `is_authenticated`, `is_active`, `is_anonymous` e `get_id()`.

-----

### 2\. Criando o Fluxo de Autenticação

Agora vamos construir as rotas e os templates para as funcionalidades de login, logout e registro de usuário.

#### Registro de Usuário

Primeiro, crie uma rota para o registro. É aqui que os novos usuários criarão suas contas. **Sempre faça hash das senhas** antes de salvá-las no banco de dados para garantir a segurança.

**`app.py`**

In [ ]:
# Rota para o registro de novos usuários
@app.route('/registro', methods=['GET', 'POST'])
def registro():
    if request.method == 'POST':
        hashed_senha = generate_password_hash(request.form.get('senha'), method='sha256')
        novo_usuario = Usuario(
            nome=request.form.get('nome'),
            email=request.form.get('email'),
            senha=hashed_senha
        )
        db.session.add(novo_usuario)
        db.session.commit()
        flash('Sua conta foi criada! Agora você pode fazer o login.', 'success')
        return redirect(url_for('login'))
    return render_template('registro.html')

**`templates/registro.html`**

```html
{% extends "layout.html" %}
{% block content %}
    <h2>Registro</h2>
    <form method="POST">
        <div>
            <label for="nome">Nome:</label>
            <input type="text" id="nome" name="nome" required>
        </div>
        <div>
            <label for="email">Email:</label>
            <input type="email" id="email" name="email" required>
        </div>
        <div>
            <label for="senha">Senha:</label>
            <input type="password" id="senha" name="senha" required>
        </div>
        <button type="submit">Registrar</button>
    </form>
{% endblock %}
```

#### Login do Usuário

A rota de login verificará as credenciais do usuário. Usamos `check_password_hash()` para comparar a senha fornecida com o hash salvo no banco de dados.

**`app.py`**

In [ ]:
# Rota para o login
@app.route('/login', methods=['GET', 'POST'])
def login():
    if current_user.is_authenticated:
        return redirect(url_for('home'))

    if request.method == 'POST':
        usuario = Usuario.query.filter_by(email=request.form.get('email')).first()
        if usuario and check_password_hash(usuario.senha, request.form.get('senha')):
            login_user(usuario)
            flash('Login realizado com sucesso!', 'success')
            next_page = request.args.get('next')
            return redirect(next_page or url_for('dashboard'))
        else:
            flash('Email ou senha incorretos.', 'error')
            return redirect(url_for('login'))

    return render_template('login.html')

**`templates/login.html`**

```html
{% extends "layout.html" %}
{% block content %}
    <h2>Login</h2>
    <form method="POST">
        <div>
            <label for="email">Email:</label>
            <input type="email" id="email" name="email" required>
        </div>
        <div>
            <label for="senha">Senha:</label>
            <input type="password" id="senha" name="senha" required>
        </div>
        <button type="submit">Entrar</button>
    </form>
{% endblock %}
```

#### Logout do Usuário

A rota de logout é simples, basta chamar a função `logout_user()` do Flask-Login para remover o usuário da sessão.

**`app.py`**

In [ ]:
# Rota para o logout
@app.route('/logout')
@login_required # Protege a rota para que apenas usuarios logados possam acessa-la
def logout():
    logout_user()
    flash('Você foi desconectado.', 'info')
    return redirect(url_for('home'))

-----

### 3\. Protegendo Rotas com `login_required`

O decorador `@login_required` do Flask-Login é a maneira mais fácil de proteger uma rota. Se um usuário tentar acessar uma rota protegida sem estar logado, ele será redirecionado para a página de login.

**`app.py`**

In [ ]:
# Rota de exemplo protegida
@app.route('/dashboard')
@login_required
def dashboard():
    return render_template('dashboard.html')

**`templates/dashboard.html`**

```html
{% extends "layout.html" %}
{% block content %}
    <h2>Dashboard</h2>
    <p>Olá, {{ current_user.nome }}! Bem-vindo ao seu painel.</p>
{% endblock %}
```

O objeto `current_user` fica disponível em todas as views e templates, permitindo que você acesse as informações do usuário logado.

-----

### 4\. Dica de Segurança

Nunca use senhas em texto puro\! Sempre faça o hash das senhas antes de salvar no banco de dados. A extensão **Werkzeug** é a ferramenta ideal para isso.

In [ ]:
# Para fazer o hash da senha:
hashed_password = generate_password_hash('senha123')

# Para verificar a senha no login:
check_password_hash(hashed_password, 'senha123')